In [4]:
# The code was removed by Watson Studio for sharing.

# IEEE CIS Fraud Detection Feature Engineering and Modeling

In [5]:
# No need to feature engineer

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from pixiedust.display import *

Pixiedust database opened successfully
Table VERSION_TRACKER created successfully
Table METRICS_TRACKER created successfully

Share anonymous install statistics? (opt-out instructions)

PixieDust will record metadata on its environment the next time the package is installed or updated. The data is anonymized and aggregated to help plan for future releases, and records only the following values:

{
   "data_sent": currentDate,
   "runtime": "python",
   "application_version": currentPixiedustVersion,
   "space_id": nonIdentifyingUniqueId,
   "config": {
       "repository_id": "https://github.com/ibm-watson-data-lab/pixiedust",
       "target_runtimes": ["Data Science Experience"],
       "event_id": "web",
       "event_organizer": "dev-journeys"
   }
}
You can opt out by calling pixiedust.optOut() in a new cell.


Pixiedust runtime updated. Please restart kernel
Table SPARK_PACKAGES created successfully
Table USER_PREFERENCES created successfully
Table service_connections created successfully


In [7]:
# The code was removed by Watson Studio for sharing.

In [8]:
# Feature engineering for training
df = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('transaction_id_train_nonull.csv', 'default-donotdelete-pr-l16zufkrkht8vm'))

In [10]:
# place transaction id and target as first and second columns
cols = df.columns
a = ['TransactionID', 'isFraud']
cols = a + [i for i in cols if i not in a]
df2 = df.select(cols)

In [ ]:
display(df2)

## Modeling

### Machine Learning

In [11]:
splits = df2.randomSplit([0.8, 0.2])
df_train = splits[0]
df_test = splits[1]

In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.mllib.evaluation import MulticlassMetrics

In [36]:
# categorical data to one-hot encode
# get categorical and numerical columns
cat = ['id_33', 'id_30', 'id_34', 'DeviceInfo', 'id_16', 
       'P_emaildomain', 'R_emaildomain', 'id_31', 'DeviceType', 
       'id_29', 'id_28', 'id_38', 'id_35', 'id_37', 'id_15',
      'id_36', 'card4', 'card6', 'ProductCD', 'id_12'] 
num = [c for c in df2.columns if c not in cat and c not in ['TransactionID', 'isFraud']]
target = ['isFraud']

In [37]:
# get one hot encoded vector names
catVec = [s+'Vec' for s in cat]

In [38]:
indexers = [StringIndexer(inputCol=s, outputCol=s+'Index') for s in target + num]

In [39]:
encoder = OneHotEncoderEstimator(inputCols=cat, outputCols=catVec)

In [46]:
vectorAssembler = VectorAssembler(inputCols=num + catVec,
                                  outputCol="features")

scaler = StandardScaler(inputCol="features", outputCol="features_scaled")

# PCA

In [47]:
from pyspark.ml.classification import GBTClassifier

gbt = GBTClassifier(labelCol="label", featuresCol="features_scaled", maxIter=10)

In [48]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=indexers + [encoder, vectorAssembler, scaler,gbt])


In [57]:
df_train.groupBy('isFraud').count().show()

+-------+------+
|isFraud| count|
+-------+------+
|    1.0| 16538|
|    0.0|455662|
+-------+------+



In [64]:
16538/455662

0.0362944463220545

In [71]:
df_train.dtypes

[('TransactionID', 'string'),
 ('isFraud', 'string'),
 ('D13', 'string'),
 ('D14', 'string'),
 ('D12', 'string'),
 ('id_04', 'string'),
 ('id_03', 'string'),
 ('D6', 'string'),
 ('D9', 'string'),
 ('D8', 'string'),
 ('id_09', 'string'),
 ('id_10', 'string'),
 ('id_32', 'string'),
 ('id_14', 'string'),
 ('V153', 'string'),
 ('V161', 'string'),
 ('V162', 'string'),
 ('V163', 'string'),
 ('V138', 'string'),
 ('V157', 'string'),
 ('V156', 'string'),
 ('V155', 'string'),
 ('V154', 'string'),
 ('V139', 'string'),
 ('V149', 'string'),
 ('V158', 'string'),
 ('V147', 'string'),
 ('V146', 'string'),
 ('V142', 'string'),
 ('V141', 'string'),
 ('V140', 'string'),
 ('V148', 'string'),
 ('V152', 'string'),
 ('V151', 'string'),
 ('V150', 'string'),
 ('V145', 'string'),
 ('V144', 'string'),
 ('V143', 'string'),
 ('V159', 'string'),
 ('V160', 'string'),
 ('V165', 'string'),
 ('V164', 'string'),
 ('V166', 'string'),
 ('V322', 'string'),
 ('V332', 'string'),
 ('V339', 'string'),
 ('V338', 'string'),
 ('V

In [72]:
# sample

fractions = {'0.0': 0.036, '1.0': 1.}
df_train_sample = df_train.sampleBy('isFraud', fractions=fractions, seed=42)
df_train_sample.groupBy('isFraud').count().show()

+-------+-----+
|isFraud|count|
+-------+-----+
|    1.0|16538|
|    0.0|16416|
+-------+-----+



In [74]:
# save the sample
# save
path = "cos://default-donotdelete-pr-l16zufkrkht8vm.os_938e241641654cf0a8c1bb0aefbea33d_configs/df_train_sample.csv"
df_train_sample.write.format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
    .option('header', 'true').save(path)

In [75]:
model = pipeline.fit(df_train_sample)

KeyboardInterrupt: 

In [ ]:
model.save("gbt1")

In [ ]:
# F1 score on train

In [ ]:
# prediction = model.transform(df_train)

In [ ]:
from pyspark.sql.functions import arrays_zip

In [ ]:
predictionAndLabels = arrays_zip(prediction, df_train.isFraud)

In [ ]:
predictionAndLabels = df_train.map(lambda lp: (model.predict(lp), lp.isFraud))

In [ ]:
# Instantiate metrics object
metrics = MulticlassMetrics(predictionAndLabels)

# Overall statistics
precision = metrics.precision()
recall = metrics.recall()
f1Score = metrics.fMeasure()
print("Summary Stats")
print("Precision = %s" % precision)
print("Recall = %s" % recall)
print("F1 Score = %s" % f1Score)


In [ ]:
# F1 score on test

### Save model

### Hyperparameter Tuning using Cross Validation

### Deep Learning

In [25]:
!pip install systemml
!pip install --upgrade https://github.com/niketanpansare/future_of_data/raw/master/systemml-1.3.0-SNAPSHOT-python.tar.gz

  Using cached https://files.pythonhosted.org/packages/29/50/a552a5aff252ae915f522e44642bb49a7b7b31677f9580cfd11bcc869976/scipy-1.3.1-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/a0/c5/d2238762d780dde84a20b8c761f563fe882b88c5a5fb03c056547c442a19/scikit_learn-0.21.3-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/e5/e6/c3fdc53aed9fa19d6ff3abf97dfad768ae3afce1b7431f7500000816bda5/numpy-1.17.2-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/73/9b/52e228545d14f14bb2a1622e225f38463c8726645165e1cb7dde95bfe6d4/pandas-0.25.1-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/14/41/db6dec65ddbc176a59b89485e8cc136a433ed9c6397b6bfe2cd38412051e/Pillow-6.1.0-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/8f/42/155696f85f344c066e17af287359c9786b436b1bf86029bb3411283274f3/joblib-0.14.0-py2.py3-none-any.

In [23]:
from systemml import MLContext, dml

In [26]:
from systemml.mllearn import Keras2DML

In [21]:
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout

In [ ]:
input_shape
sysml_model = Keras2DML(spark, keras_model, input_shape=input_shape, weights='weights_dir')
sysml_model.setConfigProperty('sysml.native.blas', 'openblas')
sysml_model.setConfigProperty('sysml.native.blas.directory', os.path.join(os.getcwd(),'OpenBLAS-0.2.20/'))
sysml_model.summary()
sysml_model.fit(X_train, y_train)

### Save model

In [ ]:
save(sc, path)